# More Tests

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
from brian2 import *

In [3]:
# functions for plotting
def plot_output_colored(t, v, u, I, save_fig=False, file_name='test.png'):
    fig, axes = plt.subplots(4, sharey=True, figsize=(12, 12))
    
    axes[0].plot(t, v, label='v')
    axes[0].plot(t, u, label='u')
    axes[0].plot(t, I, label='I')
    axes[0].axhline(30, ls='-', c='C3', lw=2, label='v=30')
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('v, u, I')
    axes[0].set_title('Trimodal')
    axes[0].legend()
    
    axes[1].plot(t, v, label='v')
    axes[1].plot(t, u, label='u')
    axes[1].plot(t, I, label='I')
    axes[1].axhline(30, ls='-', c='C3', lw=2, label='v=30')
    axes[1].set_xlim([1000, 2000])
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('v, u, I')
    axes[1].set_title('Tonic')
    axes[1].legend()
    
    axes[2].plot(t, v, label='v')
    axes[2].plot(t, u, label='u')
    axes[2].plot(t, I, label='I')
    axes[2].axhline(30, ls='-', c='C3', lw=2, label='v=30')
    axes[2].set_xlim([7000, 8000])
    axes[2].set_xlabel('Time (ms)')
    axes[2].set_ylabel('v, u, I')
    axes[2].set_title('Burst')
    axes[2].legend()
    
    axes[3].plot(t, v, label='v')
    axes[3].plot(t, u, label='u')
    axes[3].plot(t, I, label='I')
    axes[3].axhline(30, ls='-', c='C3', lw=2, label='v=30')
    axes[3].set_xlim([13000, 14000])
    axes[3].set_xlabel('Time (ms)')
    axes[3].set_ylabel('v, u, I')
    axes[3].set_title('Quiescent')
    axes[3].legend()
    
    fig.tight_layout()
    
    if save_fig:
        plt.savefig('outputs/' + file_name, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()
    
    
def plot_output(t, v, u, save_fig=False, file_name='test.png'):
    fig, axes = plt.subplots(4, sharey=True, figsize=(12, 12))
    
    axes[0].plot(t, v, 'k-', label='v')
    axes[0].plot(t, u, 'k:', label='u')
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('v, u')
    axes[0].set_title('Trimodal')
    axes[0].legend()
    
    axes[1].plot(t, v, 'k-', label='v')
    axes[1].plot(t, u, 'k:', label='u')
    axes[1].set_xlim([1000, 2000])
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('v, u')
    axes[1].set_title('Tonic')
    axes[1].legend()
    
    axes[2].plot(t, v, 'k-', label='v')
    axes[2].plot(t, u, 'k:', label='u')
    axes[2].set_xlim([7000, 8000])
    axes[2].set_xlabel('Time (ms)')
    axes[2].set_ylabel('v, u')
    axes[2].set_title('Burst')
    axes[2].legend()
    
    axes[3].plot(t, v, 'k-', label='v')
    axes[3].plot(t, u, 'k:', label='u')
    axes[3].set_xlim([13000, 14000])
    axes[3].set_xlabel('Time (ms)')
    axes[3].set_ylabel('v, u')
    axes[3].set_title('Quiescent')
    axes[3].legend()
    
    fig.tight_layout()
    
    if save_fig:
        plt.savefig('outputs/' + file_name, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()
    
    
def plot_output_simple(t, v, u, save_fig=False, file_name='test.png'):
    plt.figure(figsize=(12, 4))
    plt.plot(t, v, 'k-', label='v')
    plt.plot(t, u, 'k:', label='u')
    plt.xlabel('Time (ms)')
    plt.ylabel('v, u')
    plt.title('Trimodal')
    plt.legend()
    
    if save_fig:
        plt.savefig('outputs/' + file_name, bbox_inches='tight')
        plt.close()
    else:
        plt.show()
    
    

### Alcohol Inhibition (Complex, Modified 2)

In [4]:
# total run duration 90s
# desired quiescent 10s
# desired bimodal 25s
# desired tonic 15s

def IZH_inh_modified2(
    fI='int(t>100*ms)*10', 
    V=-65, 
    tau=0.25, 
    duration_t=5000,
    duration_b=7000,
    duration_q=8000,
    plot_colored=False, #True,
    e_in=0, 
    e_max=0.45,
    Ra=0,
    Rb=0,
    RI=0,
    save_output=False,
    output_file='test.png',
):
        
    defaultclock.dt = tau*ms
    tau = tau/ms
    
    eqs = '''
    dv/dt = int(t>duration/10)*tau*(0.04*v**2+5*v+140-u+((1-RI*(e_in/e_max))*I)) : 1
    du/dt = int(t>duration/10)*tau*((1-Ra*(e_in/e_max))*a)*(((1-Rb*(e_in/e_max))*b)*v-u) : 1
    I : 1
    '''

    # Create a NeuronGroup with one neuron using previous equations
    G = NeuronGroup(1, eqs, threshold='v>=30', reset='v=c; u+=d', method='euler')
    G.v = V
    G.u = 0.2*V
    
    M = StateMonitor(G, ('v', 'u', 'I'), record=0)
    S = SpikeMonitor(G)
    
    @network_operation(dt=1*ms)
    def change_I():
        G.I = fI
    
    print(f'Running model...')
    a = 0.02
    b = 0.2
    c = -65
    d = 0.5 # ISI_tonic = 30ms
    
    Ra = 0.2
    Rb = 0.6
    RI = 0.4
    duration = 10000*ms
    run(duration)
    
    Ra = 0.1
    Rb = 0.4
    RI = 0.3
    duration = 10000*ms
    run(duration)
    
    Ra = 0.2
    Rb = 0.6
    RI = 0.4
    duration = 10000*ms
    run(duration)
    
    Ra = 0.1
    Rb = 0.4
    RI = 0.3
    duration = 20000*ms
    run(duration)
    
    Ra = 0.15
    Rb = 0.5
    RI = 0.35
    duration = 10000*ms
    run(duration)
    
    Ra = 0.2
    Rb = 0.6
    RI = 0.4
    duration = 20000*ms
    run(duration)
   
    # Plotting
    """
    plt.figure()
    plt.plot(M.t/ms, M.Ra[0], 'k-', label='Ra')
    plt.plot(M.t/ms, M.Rb[0], 'k--', label='Rb')
    plt.plot(M.t/ms, M.RI[0], 'k:', label='RI')
    plt.xlabel('Time')
    plt.ylabel('Ratio')
    plt.legend()
    plt.show()
    """
    
    """
    print('Creating plot.')
    if plot_colored:
        plot_output_colored(M.t/ms, M.v[0], M.u[0], M.I[0], save_fig=save_output, file_name=output_file)
    else:
        plot_output_simple(M.t/ms, M.v[0], M.u[0], save_fig=save_output, file_name=output_file)
    """
        
    return(np.diff(S.t))
    


In [5]:
inh_vals = [0.05, 0.10, 0.15, 0.20, 0.30, 0.35, 0.40, 0.45]
ISIs = []

for val in inh_vals:
    print(f'Testing e = {val}')
    ISI = IZH_inh_modified2(e_in=val, save_output=False)
    ISIs.append(ISI)
    print('')
    
    

Testing e = 0.05
Running model...


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8, 3))

av_ISIs = []
for ISI in ISIs:
    av_ISIs.append(np.average(ISI))

plt.scatter(inh_vals, av_ISIs, color='k', marker='s')
plt.plot(inh_vals[:-2], av_ISIs[:-2], 'k--')
plt.xlim([0, None])
plt.xlabel('BAC (%)')
plt.ylabel('Average ISI (s)')

plt.savefig('outputs/final_model_avg_isi.png', bbox_inches='tight')
plt.show()
plt.close()